In [6]:
using JSON

# Specify the file name and location
filename = "tiny.json"

# Read the JSON file
data = JSON.parsefile(filename)

# Extract the details from the JSON
# For example, extract all shops
linear_model = true  # Set to false if you don't want a linear model
shops = data["shops"]
println("Shops:")
println(shops)

# Extract vehicles
vehicles = data["vehicles"]
shops = data["shops"]
parameters = data["parameters"]
constraints = data["constraints"]

for vehicle in vehicles
    if linear_model
        # Set all vehicles to regular if linear_model is true
        println("Vehicle Type: regular")
    else
        # Otherwise, print the actual vehicle type
        println("Vehicle ID: ", vehicle["id"], ", Type: ", vehicle["type"])
    end
end
for constraint in constraints
    println("Constraint: ", constraint)
end
for parameter in parameters
    println("Parameter: ", parameter)
end
for shop in shops
    println("Shop: ", shop)
end



Shops:
Any[Dict{String, Any}("name" => "body", "resequencing_lag" => 1), Dict{String, Any}("name" => "paint", "resequencing_lag" => 2), Dict{String, Any}("name" => "assembly", "resequencing_lag" => 0)]
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Constraint: Dict{String, Any}("max_vehicles" => 4, "cost" => 2, "min_vehicles" => 2, "id" => 1, "vehicles" => Any[1, 2, 4], "type" => "batch_size", "shop" => "body")
Constraint: Dict{String, Any}("cost" => 3, "partition" => Any[Any[1, 2], Any[3, 4, 5]], "id" => 2, "type" => "lot_change", "shop" => "paint")
Constraint: Dict{String, Any}("max_vehicles" => 2, "cost" => 2, "id" => 3, "window_size" => 3, "vehicles" => Any[1, 2, 5], "type" => "rolling_window", "shop" => "assembly")
Constraint: Dict{String, Any}("max_vehicles" => 3, "cost" => 3, "min_vehicles" => 2, "id" => 4, "vehicles" => Any[1, 2, 3], "type" => "batch_size", "shop" => "assembly")
Constraint: Dict{String, Any}("max_ve

In [ ]:
resequencing_cost = parameters["resequencing_cost"]
shops ={}
for shop in shops 
    
end 
lot_changeover_cost = 0
rolling_window_cost = 0 
batch_size_cost = 0

for constraint in constraints
    
end

20

In [ ]:
d